In [3]:
import pandas as pd
import numpy as np
from rapidfuzz import process, utils, fuzz

In [8]:
data=pd.read_csv('C:/wamp/tmp/CSVS_2/recommendations_master_sheet_1 - Copy.csv')
df=pd.DataFrame(data)
required_cols=['id','age','location','gender','reason','source_type','skin_type','product_type','product_name']
#df[required_cols].tail(10)

In [9]:
sorted_df = df.sort_values(by="product_name")
sorted_df.fillna('', inplace=True)

In [10]:
sorted_df_list = sorted_df.values.tolist()
sorted_df = sorted_df.reset_index(drop=True)
unique_product_names = sorted_df['product_name'].unique().tolist()
#unique_product_names

In [13]:
all_sorted_df_list = sorted_df['product_name'].tolist()
all_sorted_df_list

['.',
 '1% Hyaluronic Sunscreen Aqua Gel',
 '1% Hyaluronic Sunscreen Aqua Gel from The derma',
 '1% Salicylic Acid Oil-Free Moisturizer For Face with Oat Extract',
 '1% Salicylic Acid Oil-Free Moisturizer For Face with Oat Extract.',
 '1% derma salisaylic acid face wash',
 '1% hyaluronic sunscreen aqualogica',
 '1. La Roche Posay Sunscreen\n2. Aqualogica Detan + Dewy Sunscreen with Cherry Tomato and Hyaluronic Acid',
 '1derma co. 2 lotus 3 ayuga',
 '2%Salicylic acid face wash',
 '72 hours dot and key hydrating moisturiser',
 '82E Turmeric Shield serum sunscreen',
 '82e',
 'A derma face wash',
 'A ret gel.',
 'A- ret',
 'A-Derma Epithelial AH moisturiser',
 'ABnext gel',
 'ACNE OC',
 'ACNE UV',
 'ACNE UV GEL SPF 30',
 'ACmist cream',
 'ADERMA hydrating',
 'ADHYL face serum',
 'AGELITE FACEWASH',
 'AHA',
 'AHA',
 'AHA',
 'AHA BHA derma co',
 'AHA BHA peel COSRX',
 'AHA GLOW S face wash',
 'AHA Glow FW',
 'AHA Glow Face Wash',
 'AHA Glow Face Wash',
 'AHA Glow Face Wash',
 'AHA Glow Face 

In [12]:
processed_products = [utils.default_process(product) for product in all_sorted_df_list]
processed_products

['',
 '1  hyaluronic sunscreen aqua gel',
 '1  hyaluronic sunscreen aqua gel from the derma',
 '1  salicylic acid oil free moisturizer for face with oat extract',
 '1  salicylic acid oil free moisturizer for face with oat extract',
 '1  derma salisaylic acid face wash',
 '1  hyaluronic sunscreen aqualogica',
 '1  la roche posay sunscreen 2  aqualogica detan   dewy sunscreen with cherry tomato and hyaluronic acid',
 '1derma co  2 lotus 3 ayuga',
 '2 salicylic acid face wash',
 '72 hours dot and key hydrating moisturiser',
 '82e turmeric shield serum sunscreen',
 '82e',
 'a derma face wash',
 'a ret gel',
 'a  ret',
 'a derma epithelial ah moisturiser',
 'abnext gel',
 'acne oc',
 'acne uv',
 'acne uv gel spf 30',
 'acmist cream',
 'aderma hydrating',
 'adhyl face serum',
 'agelite facewash',
 'aha',
 'aha',
 'aha',
 'aha bha derma co',
 'aha bha peel cosrx',
 'aha glow s face wash',
 'aha glow fw',
 'aha glow face wash',
 'aha glow face wash',
 'aha glow face wash',
 'aha glow face wash

In [14]:
from rapidfuzz.process import cdist

def find_matched_product():
    # Calculate distance between all the names
    sa = cdist(processed_products, processed_products,score_cutoff=80, workers=-1)
    matches_list = []
    for distances in sa:
        indices = np.argwhere(~np.isin(distances, [100, 0])).flatten()
        names = list(map(processed_products.__getitem__, indices))
        matches_list.append(names)
    dfff = pd.DataFrame({'second_df_names_of_product': processed_products, 'matches': matches_list})
    dfff['matches_count'] = dfff.matches.str.len()
    return dfff

In [15]:
dfff=find_matched_product()

In [17]:
df_merge = pd.concat([sorted_df,dfff], axis=1)

In [24]:
#df_merge.head(70)

In [22]:
best_matches=[]
for ind in df_merge.index:
    matches_for_product_list=df_merge['matches'][ind]
    if len(matches_for_product_list)==0:
        best_matches.append(df_merge['second_df_names_of_product'][ind])
    else:
        max_len_match = get_max_length_match(matches_for_product_list)
        best_matches.append(max_len_match)
df_merge['best_match']=best_matches

In [21]:
def get_max_length_match(match):
    max_len=0
    max_len_item=""
    for item in match:
        if(len(item)>max_len):
            max_len=len(item)
            max_len_item=item
    return max_len_item

In [23]:
df_merge.head(80)

,id,age,location,gender,reason,source_type,skin_type,product_type,product_name,second_df_names_of_product,matches,matches_count,best_match
0,wcqniezgn4rj1fqlth3i5wcqnilff4rd,22,-,Female,.,USER,Oily,sunscreen,.,,[],0,
1,xsj7vammhc6wmvbyb7xsj7vambdan5bm,24,Kota,Female,-,INF,Oily,sunscreen,1% Hyaluronic Sunscreen Aqua Gel,1 hyaluronic sunscreen aqua gel,[1 hyaluronic sunscreen aqua gel from the der...,12,1 hyaluronic sunscreen aqua gel from the derma
2,2gt1f6l88anwvh82gtnnvmho113p77h7,19,Agartala tripura,Female,-,INF,Oily,sunscreen,1% Hyaluronic Sunscreen Aqua Gel from The derma,1 hyaluronic sunscreen aqua gel from the derma,[1 hyaluronic sunscreen aqua gel],1,1 hyaluronic sunscreen aqua gel
3,2gt1f6l88anwvh82gtnnvmho113p77h7,19,Agartala tripura,Female,-,INF,Oily,moisturiser,1% Salicylic Acid Oil-Free Moisturizer For Fac...,1 salicylic acid oil free moisturizer for fac...,[],0,1 salicylic acid oil free moisturizer for fac...
4,wwx7hcup42hpm1lvwwx7g1l99273oybc,30,Mumbai,Female,-,INF,Oily,moisturiser,1% Salicylic Acid Oil-Free Moisturizer For Fac...,1 salicylic acid oil free moisturizer for fac...,[],0,1 salicylic acid oil free moisturizer for fac...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,n0vebasagynx3wvn0vedyerfrs4fydyl,0,Mumbai,-,-,DERM,oily,cleanser,AHA glow face wash,aha glow face wash,"[aha glow s face wash, aha glow facewash, aha ...",170,aha glow s facewash
76,rrszrlkkuikgm8gku7rrszrlqo67pipp,0,Hyderabad,-,-,DERM,combination,cleanser,AHA glow face wash,aha glow face wash,"[aha glow s face wash, aha glow facewash, aha ...",170,aha glow s facewash
77,41bqirx92mfbshuocvh41bqiwrrs205h,0,Ghaziabad,-,-,DERM,combination,cleanser,AHA glow face wash,aha glow face wash,"[aha glow s face wash, aha glow facewash, aha ...",170,aha glow s facewash
78,2fiiuiqa592umg4bbd5ua52fiiuiq8ja,0,Bangalore,-,-,DERM,combination,cleanser,AHA glow facewash,aha glow facewash,"[aha glow s face wash, aha glow face wash, aha...",189,aha glow s facewash
